In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
data_dir = 'data'

folder, dirs, filenames = next(os.walk(data_dir))
filenames = [os.path.join(data_dir, name) for name in filenames] 
filenames[:5], len(filenames)

In [ ]:
def get_vocabulary_size(filenames): 
    for filename in filenames:
        doc = []
        with open(filename, 'r') as f:
            for row in f:
                doc.append(row.lower())
            doc = " ".join(doc).split()
    return doc
n_vocab = len(set(get_vocabulary_size(train_files)))

In [ ]:
def generate_window_dataset(filenames, window_size=10, shuffle=False, batch_size=32):
    documents = []
    for f in filenames:
        doc = tf.io.read_file(f)
        doc = tf.strings.lower(doc) # convert to lower case
        doc = tf.strings.regex_replace(doc,"\n"," ") # replacing new line with space
        doc = tf.strings.regex_replace(doc, "\r", "") # replcaing \r charter
        doc = tf.strings.split(doc, ' ') # split to word level 
        documents = documents + doc.numpy().tolist() # make dict word all file
    doc_dataset = tf.data.Dataset.from_tensor_slices(documents) # load data into tensor from
    
    # make generate windows from list documents 
    # E.x. ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    doc_dataset = doc_dataset.window(size=window_size + 1, shift=1, drop_remainder=True)
    
    # E.x. [['A', 'B', 'C', 'D', 'E', 'F', 'G']]
    doc_dataset = doc_dataset.flat_map(lambda x: x.batch(window_size + 1))
    
    # E.x. ['A', 'B', 'C', 'D', 'E', 'F'] => [G]    
    #      ['B', 'C', 'D', 'E', 'F', 'G'] => [K]
    doc_dataset = doc_dataset.map(lambda x: (x[:-1] , x[-1:]))
    
    # shuffle the dataset with window size
    if shuffle:
        doc_dataset = doc_dataset.shuffle(len(list(doc_dataset)))
    
    doc_dataset = doc_dataset.batch(batch_size)
    doc_dataset = doc_dataset.prefetch(batch_size)
    
    return doc_dataset , documents
